In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
import os
from time import time
import datetime
from keras.callbacks import TensorBoard
import tensorflow as tf
from keras import backend as K
import multiprocessing as mp


Using TensorFlow backend.


In [2]:
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

In [3]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# CPU Trial

In [10]:
num_CPU=mp.cpu_count()

config = tf.ConfigProto(intra_op_parallelism_threads=8,\
        inter_op_parallelism_threads=8, allow_soft_placement=True,\
        device_count = {'CPU' : num_CPU, 'GPU' : 0})
session = tf.Session(config=config)
K.set_session(session)

In [11]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [12]:
start = time()

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True, callbacks=[tensorboard])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4,
                        steps_per_epoch=len(x_train)/batch_size, 
                        callbacks=[tensorboard])

Using real-time data augmentation.
Epoch 1/100
1563/1562 [==============================] - 84s 53ms/step - loss: 1.8641 - acc: 0.3136 - val_loss: 1.6171 - val_acc: 0.4167
Epoch 2/100
1563/1562 [==============================] - 83s 53ms/step - loss: 1.5847 - acc: 0.4200 - val_loss: 1.4007 - val_acc: 0.4932
Epoch 3/100
1563/1562 [==============================] - 83s 53ms/step - loss: 1.4554 - acc: 0.4722 - val_loss: 1.2790 - val_acc: 0.5428
Epoch 4/100
1563/1562 [==============================] - 83s 53ms/step - loss: 1.3749 - acc: 0.5054 - val_loss: 1.2868 - val_acc: 0.5374
Epoch 5/100
1563/1562 [==============================] - 83s 53ms/step - loss: 1.3017 - acc: 0.5340 - val_loss: 1.1565 - val_acc: 0.5926
Epoch 6/100
1563/1562 [==============================] - 83s 53ms/step - loss: 1.2441 - acc: 0.5614 - val_loss: 1.1990 - val_acc: 0.5744
Epoch 7/100
1563/1562 [==============================] - 83s 53ms/step - loss: 1.1990 - acc: 0.5740 - val_loss: 1.0740 - val_acc: 0.6139
Epoch 

1563/1562 [==============================] - 83s 53ms/step - loss: 0.7858 - acc: 0.7343 - val_loss: 0.7231 - val_acc: 0.7625
Epoch 61/100
1563/1562 [==============================] - 83s 53ms/step - loss: 0.7842 - acc: 0.7380 - val_loss: 0.6993 - val_acc: 0.7700
Epoch 62/100
1563/1562 [==============================] - 83s 53ms/step - loss: 0.7778 - acc: 0.7394 - val_loss: 0.7269 - val_acc: 0.7623
Epoch 63/100
1563/1562 [==============================] - 83s 53ms/step - loss: 0.7870 - acc: 0.7336 - val_loss: 0.7494 - val_acc: 0.7619
Epoch 64/100
1563/1562 [==============================] - 83s 53ms/step - loss: 0.7872 - acc: 0.7349 - val_loss: 0.7191 - val_acc: 0.7585
Epoch 65/100
1563/1562 [==============================] - 83s 53ms/step - loss: 0.7913 - acc: 0.7371 - val_loss: 0.6973 - val_acc: 0.7671
Epoch 66/100
1563/1562 [==============================] - 83s 53ms/step - loss: 0.7831 - acc: 0.7373 - val_loss: 0.7455 - val_acc: 0.7464
Epoch 67/100
1563/1562 [=======================

In [13]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)

end = time()
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
print('CPU Runtime:', str(end-start))

10000/10000 [==============================] - 4s 400us/step
Test loss: 0.7958948613166809
Test accuracy: 0.7422
CPU Runtime: 8309.377392053604


# GPU Trial

In [6]:
# num_CPU=mp.cpu_count()

# config = tf.ConfigProto(intra_op_parallelism_threads=8,\
#         inter_op_parallelism_threads=8, allow_soft_placement=True,\
#         device_count = {'CPU' : num_CPU, 'GPU' : 1})
# session = tf.Session(config=config)
# K.set_session(session)

K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [20]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [8]:
start = time()

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True, callbacks=[tensorboard])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4,
                        steps_per_epoch=len(x_train)/batch_size,
                        callbacks=[tensorboard])

Using real-time data augmentation.
Epoch 1/100
1563/1562 [==============================] - 13s 8ms/step - loss: 0.7820 - acc: 0.7376 - val_loss: 0.7222 - val_acc: 0.7625
Epoch 2/100
1563/1562 [==============================] - 13s 8ms/step - loss: 0.7808 - acc: 0.7372 - val_loss: 0.7156 - val_acc: 0.7605
Epoch 3/100
1563/1562 [==============================] - 13s 8ms/step - loss: 0.7805 - acc: 0.7383 - val_loss: 0.8211 - val_acc: 0.7446
Epoch 4/100
1563/1562 [==============================] - 13s 8ms/step - loss: 0.7740 - acc: 0.7396 - val_loss: 0.7718 - val_acc: 0.7432
Epoch 5/100
1563/1562 [==============================] - 13s 8ms/step - loss: 0.7802 - acc: 0.7393 - val_loss: 0.7278 - val_acc: 0.7575
Epoch 6/100
1563/1562 [==============================] - 13s 9ms/step - loss: 0.7782 - acc: 0.7409 - val_loss: 0.7980 - val_acc: 0.7400
Epoch 7/100
1563/1562 [==============================] - 13s 8ms/step - loss: 0.7780 - acc: 0.7365 - val_loss: 0.6632 - val_acc: 0.7816
Epoch 8/100
1

1563/1562 [==============================] - 13s 8ms/step - loss: 0.8858 - acc: 0.7107 - val_loss: 1.0037 - val_acc: 0.7114
Epoch 61/100
1563/1562 [==============================] - 13s 8ms/step - loss: 0.8987 - acc: 0.7081 - val_loss: 0.7524 - val_acc: 0.7475
Epoch 62/100
1563/1562 [==============================] - 13s 8ms/step - loss: 0.9054 - acc: 0.7064 - val_loss: 0.7910 - val_acc: 0.7491
Epoch 63/100
1563/1562 [==============================] - 13s 8ms/step - loss: 0.9036 - acc: 0.7047 - val_loss: 0.7989 - val_acc: 0.7443
Epoch 64/100
1563/1562 [==============================] - 13s 8ms/step - loss: 0.9074 - acc: 0.7054 - val_loss: 0.7762 - val_acc: 0.7445
Epoch 65/100
1563/1562 [==============================] - 13s 8ms/step - loss: 0.9161 - acc: 0.7051 - val_loss: 0.8269 - val_acc: 0.7328
Epoch 66/100
1563/1562 [==============================] - 13s 9ms/step - loss: 0.9127 - acc: 0.7041 - val_loss: 0.9134 - val_acc: 0.7040
Epoch 67/100
1563/1562 [==============================

In [9]:
model_name = 'gpu_keras_cifar10_trained_model.h5'

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)

end = time()
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
print('GPU Runtime:', str(end-start))

10000/10000 [==============================] - 1s 140us/step
Test loss: 1.0554213806152344
Test accuracy: 0.6747
GPU Runtime: 1322.6027481555939
